In [18]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pymysql
import re
from urllib.parse import urlparse, parse_qs

# mysql db connect
# need to fill below host, user, password
#db = pymysql.connect(host='', user='', password='', port=3306, db='readers', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()

In [2]:
# ------ 연재 웹툰 크롤링
# 요일 탭 클릭
def setDay(dayvalue):
    day = browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div[3]/div[2]/ul/li[" + str(dayvalue) + "]")
    day.click()
    browser.implicitly_wait(3)

def day_webtoon(day, num):
    webtoon = browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div[3]/div[3]/ul/li[" + str(num) + "]/a")
    
    # 작품 불러오기
    toon_url = webtoon.get_attribute('href')
    
    # ID
    toon_id = 'kakao_' + str(toon_url).replace("https://page.kakao.com/home?seriesId=", '')
    
    # 요일 추출
    if day == 1: weekday = 'mon'
    if day == 2: weekday = 'tue'
    if day == 3: weekday = 'wed'
    if day == 4: weekday = 'thu'
    if day == 5: weekday = 'fri'
    if day == 6: weekday = 'sat'
    if day == 7: weekday = 'sun'
    cursor1.execute(weekday_sql, (toon_id, weekday))
    
    # 제목
    name = webtoon.find_element_by_class_name('jsx-3847971416')
    toon_name = name.text
    
    # 작가
    writer = webtoon.find_element_by_class_name('author')
    wrt_name = writer.text
    
    # 설명
    description = webtoon.find_element_by_class_name('captionWrap')
    desc = description.find_element_by_class_name('jsx-1134470472')
    toon_desc = desc.text
    
    # 썸네일 주소
    webtoon = browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div[3]/div[3]/ul/li[" + str(num) + "]/a/div[1]/div[1]/div/img")
    browser.execute_script("arguments[0].scrollIntoView(true);", webtoon) # Scroll to this webtoon element
                         # Scroll to bottom is "window.scrollTo(0,document.body.scrollHeight)"
    toon_thumb_url = 'https://page.kakao.com/static/common/bg_noImg.png?d0b47a2245ca6f22b0660775a3d5c27a'
        # 이미지 로딩 안 된 현상 해결
    while toon_thumb_url == 'https://page.kakao.com/static/common/bg_noImg.png?d0b47a2245ca6f22b0660775a3d5c27a':
        toon_thumb_url = webtoon.get_attribute('src')
    # Also, blank page could be "https://static-page.kakao.com/static/common/bg_noImg.png?d0b47a2245ca6f22b0660775a3d5c27a";
        
    # toon_info
    cursor1.execute(sel_sql)
    result = cursor1.fetchall()
    if (toon_id not in result):
        cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'kakao', wrt_name, 'X', toon_thumb_url))
        #cursor3.execute(genre_sql, (toon_id, genre)) # 장르 정보 없음
    db.commit()

In [3]:
sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
weekday_sql = "insert ignore into toon_weekday(toon_id, toon_weekday)\
               values (%s, %s)"
'''
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"
'''


#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')
browser.get('https://page.kakao.com/main?categoryUid=10&subCategoryUid=1000')

for day in range(1, 8):
    # Scroll to Top 안 해주면 element 분석 못함;
    # 크롤링 하는 중에 페이지 내에서 내가 스크롤 해도 갑자기 오류뜸ㅋ 예민한놈
    browser.execute_script("window.scrollTo(0,0);")
    
    setDay(day)
    print('*********************')
    print(day, 'day')
    for num in range(1, 100):
        try:
            day_webtoon(day, num)
        except:
            break;

*********************
1 day
*********************
2 day


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_53764524' for key 'PRIMARY'")
  self._do_get_result()


*********************
3 day
*********************
4 day


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_48225112' for key 'PRIMARY'")
  self._do_get_result()


*********************
5 day


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_53248339' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_51155742' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_53070165' for key 'PRIMARY'")
  self._do_get_result()


*********************
6 day


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_53956161' for key 'PRIMARY'")
  self._do_get_result()


*********************
7 day


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_48211158' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_53491650' for key 'PRIMARY'")
  self._do_get_result()


In [ ]:
# 완결 작품 정보 없음

In [19]:
# ------ 회차 정보
#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')

sel_url_sql = 'select toon_id, toon_url from toon_info where toon_site = "kakao"'
sel_dup_sql = 'select toon_id, epi_name from epi_info where toon_id = %s and epi_name = %s'
ins_epi_info_sql = 'insert ignore into epi_info(toon_id, epi_name, epi_url, epi_thumb_url, epi_date)\
                    values (%s, %s, %s, %s, %s)'

cursor1.execute(sel_url_sql)
result = cursor1.fetchall()
   
for row in result:
    toon_id = row['toon_id']
    toon_url = row['toon_url'] + '&page='
    page = 1
    #print('*********************************************')
    #print('<' + toon_id + '>')
    
    num = 1 # episode 개수 탐색
    while num < 99999:
        try:
            browser.get(toon_url+ str(page))
            episode = browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div/div/div[3]/ul/li[" + str(num) + "]")
            browser.execute_script("arguments[0].scrollIntoView(true);", episode) # Scroll to this episode
            browser.implicitly_wait(3)
            
            # 에피소드 제목
            name = episode.find_element_by_xpath('div[2]/dl/dt/p')
            epi_name = name.text
            
            # 에피소드 링크
            epi_id = episode.get_attribute('data-productid')
            epi_url = "https://page.kakao.com/viewer?productId=" + epi_id
            
            # 썸네일 링크
            thumb = episode.find_element_by_xpath("div[1]/div/div/img")
                # 이미지 로딩 안 된 현상 해결
            epi_thumb_url = 'https://page.kakao.com/static/common/bg_noImg.png?d0b47a2245ca6f22b0660775a3d5c27a'
            while "bg_noImg.png" in epi_thumb_url:
                epi_thumb_url = thumb.get_attribute('src')
            
            # 업데이트 날짜 추출
            date = episode.find_element_by_xpath("div[2]/dl/dd")
            epi_date = date.text.replace(".", "-")
            
            
            
            cursor2.execute(sel_dup_sql, (toon_id, epi_name))
            if cursor2.fetchall():
                num += 1
                continue
            else:
                cursor3.execute(ins_epi_info_sql, (toon_id, epi_name, epi_url, epi_thumb_url, epi_date))
                db.commit()
                
            num += 1
        except:
            # 다음 페이지가 유효한 지 검사
            browser.get(toon_url + str(page+1))
            num = 1
            try:
                browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div/div/div[3]/ul/li[" + str(num) + "]")
                page += 1
                continue
            except NoSuchElementException:
                break